In [1]:
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# TensorFlow & Keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import (
    EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
)

# Display settings
%matplotlib inline
sns.set_style("darkgrid")
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ All libraries imported successfully!")
print(f"\nTensorFlow version: {tf.__version__}")
print(f"Using device: CPU (GPU setup will be optimized later)")
print("\n⏱️  TRAINING TIME ESTIMATE:")
print("   - With CPU: ~1-2 hours")
print("   - With GPU (when fixed): ~20-30 minutes")
print("\n💡 Note: Model quality is the same, just slower training!")

✅ All libraries imported successfully!

TensorFlow version: 2.13.0
Using device: CPU (GPU setup will be optimized later)

⏱️  TRAINING TIME ESTIMATE:
   - With CPU: ~1-2 hours
   - With GPU (when fixed): ~20-30 minutes

💡 Note: Model quality is the same, just slower training!


LOAD CLASS MAPPING & SET PARAMETERS 

In [3]:
# Define paths
data_root = Path('C:/Users/USER/fruit_veg_mushroom_identifier/data/raw')
processed_dir = Path('C:/Users/USER/fruit_veg_mushroom_identifier/data/processed')
fruits_dir = data_root / 'Fruits'
vegetables_dir = data_root / 'Vegetables'
mushrooms_dir = data_root / 'Mushrooms'
models_dir = Path('C:/Users/USER/fruit_veg_mushroom_identifier/models')
models_dir.mkdir(parents=True, exist_ok=True)

# Load class mapping
with open(processed_dir / 'class_mapping.json', 'r') as f:
    class_mapping = json.load(f)

print("📋 CLASS MAPPING LOADED")
print(f"Total classes: {class_mapping['num_classes']}")
print(f"  Fruits: {class_mapping['categories']['Fruit']}")
print(f"  Vegetables: {class_mapping['categories']['Vegetable']}")
print(f"  Mushrooms: {class_mapping['categories']['Mushroom']}")
print(f"    - Edible: {class_mapping['mushroom_safety']['EDIBLE']}")
print(f"    - Toxic: {class_mapping['mushroom_safety']['TOXIC']}")

# Training parameters
PARAMS = {
    'num_classes': class_mapping['num_classes'],
    'img_size': (224, 224),
    'batch_size': 32,
    'epochs': 50,
    'learning_rate': 0.001,
    'validation_split': 0.15,
    'test_split': 0.15,
    'model_name': 'EfficientNetB0_FruitVegMushroom'
}

print("\n⚙️ TRAINING PARAMETERS:")
for key, value in PARAMS.items():
    print(f"  {key}: {value}")


📋 CLASS MAPPING LOADED
Total classes: 44
  Fruits: 9
  Vegetables: 15
  Mushrooms: 20
    - Edible: 10
    - Toxic: 10

⚙️ TRAINING PARAMETERS:
  num_classes: 44
  img_size: (224, 224)
  batch_size: 32
  epochs: 50
  learning_rate: 0.001
  validation_split: 0.15
  test_split: 0.15
  model_name: EfficientNetB0_FruitVegMushroom


CREATE DATA GENERATORS

In [4]:
print("\n" + "="*80)
print("📊 CREATING DATA GENERATORS")
print("="*80)

# Data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# No augmentation for validation/test (only rescaling)
val_datagen = ImageDataGenerator(rescale=1./255)

print("\n✅ Data augmentation configured:")
print("  - Rotation: ±20°")
print("  - Width shift: ±20%")
print("  - Height shift: ±20%")
print("  - Shear: 20%")
print("  - Zoom: ±20%")
print("  - Horizontal flip: Yes")



📊 CREATING DATA GENERATORS

✅ Data augmentation configured:
  - Rotation: ±20°
  - Width shift: ±20%
  - Height shift: ±20%
  - Shear: 20%
  - Zoom: ±20%
  - Horizontal flip: Yes


LOAD FRUITS DATASET 

In [16]:
print("\n" + "="*80)
print("🥬 LOADING VEGETABLES DATASET")
print("="*80)

# Vegetables have train/val/test splits
veg_train_dir = vegetables_dir / 'train'
veg_val_dir = vegetables_dir / 'validation'
veg_test_dir = vegetables_dir / 'test'

veg_train_gen = train_datagen.flow_from_directory(
    veg_train_dir,
    target_size=PARAMS['img_size'],
    batch_size=PARAMS['batch_size'],
    class_mode='categorical',
    seed=42
)

veg_val_gen = val_datagen.flow_from_directory(
    veg_val_dir,
    target_size=PARAMS['img_size'],
    batch_size=PARAMS['batch_size'],
    class_mode='categorical',
    seed=42
)

veg_test_gen = val_datagen.flow_from_directory(
    veg_test_dir,
    target_size=PARAMS['img_size'],
    batch_size=PARAMS['batch_size'],
    class_mode='categorical',
    seed=42
)

# Get actual number of classes
num_vegetables = len(veg_train_gen.class_indices)

print(f"\n✅ Vegetables loaded:")
print(f"   Train samples: {veg_train_gen.samples}")
print(f"   Val samples: {veg_val_gen.samples}")
print(f"   Test samples: {veg_test_gen.samples}")
print(f"   Classes: {list(veg_train_gen.class_indices.keys())}")
print(f"   Total classes: {num_vegetables}")

veg_classes = veg_train_gen.class_indices


🥬 LOADING VEGETABLES DATASET
Found 15000 images belonging to 15 classes.
Found 3000 images belonging to 15 classes.
Found 3000 images belonging to 15 classes.

✅ Vegetables loaded:
   Train samples: 15000
   Val samples: 3000
   Test samples: 3000
   Classes: ['Bean', 'Bitter_Gourd', 'Bottle_Gourd', 'Brinjal', 'Broccoli', 'Cabbage', 'Capsicum', 'Carrot', 'Cauliflower', 'Cucumber', 'Papaya', 'Potato', 'Pumpkin', 'Radish', 'Tomato']
   Total classes: 15


LOAD VEGETABLES DATASET

In [17]:
print("\n" + "="*80)
print("🧠 BUILDING EFFICIENTNETB0 MODEL (Vegetables: 15 classes)")
print("="*80)

# Load pre-trained EfficientNetB0
base_model = EfficientNetB0(
    input_shape=(PARAMS['img_size'][0], PARAMS['img_size'][1], 3),
    weights='imagenet',
    include_top=False
)

# Freeze base model layers
base_model.trainable = False

print(f"\n✅ EfficientNetB0 base model loaded")
print(f"   Base layers: {len(base_model.layers)}")

# Build full model for vegetables (15 classes)
model = keras.Sequential([
    layers.Input(shape=(PARAMS['img_size'][0], PARAMS['img_size'][1], 3)),
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(512, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(256, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    layers.Dense(num_vegetables, activation='softmax')  # ← 15 classes, not 44
])

print(f"\n✅ Full model built successfully")
print(f"   Input: {PARAMS['img_size'] + (3,)}")
print(f"   Output: ({num_vegetables},)")

# Compile model
optimizer = Adam(learning_rate=PARAMS['learning_rate'])
model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print(f"\n✅ Model compiled")
print(f"   Loss: categorical_crossentropy")
print(f"   Metrics: accuracy")

# Print summary
print(f"\n📊 Model Summary:")
model.summary()



🧠 BUILDING EFFICIENTNETB0 MODEL (Vegetables: 15 classes)

✅ EfficientNetB0 base model loaded
   Base layers: 238

✅ Full model built successfully
   Input: (224, 224, 3)
   Output: (15,)

✅ Model compiled
   Loss: categorical_crossentropy
   Metrics: accuracy

📊 Model Summary:
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb0 (Functional  (None, 7, 7, 1280)        4049571   
 )                                                               
                                                                 
 global_average_pooling2d_1  (None, 1280)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_3 (Dense)             (None, 512)               655872    
                                                                 
 batch_normalization_2 (Bat  (None, 512

LOAD MUSHROOMS DATASET


In [18]:
print("\n" + "="*80)
print("⚙️ DEFINING TRAINING CALLBACKS")
print("="*80)

callbacks = [
    EarlyStopping(
        monitor='val_loss',
        patience=5,
        verbose=1,
        restore_best_weights=True
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        verbose=1,
        min_lr=1e-7
    ),
    ModelCheckpoint(
        filepath=str(models_dir / 'vegetables_model_best.h5'),
        monitor='val_accuracy',
        mode='max',
        save_best_only=True,
        verbose=1
    )
]

print("\n✅ Callbacks configured:")
print("  ✓ Early stopping (patience=5)")
print("  ✓ Reduce LR on plateau")
print("  ✓ Model checkpoint (save best)")



⚙️ DEFINING TRAINING CALLBACKS

✅ Callbacks configured:
  ✓ Early stopping (patience=5)
  ✓ Reduce LR on plateau
  ✓ Model checkpoint (save best)


In [19]:
print("\n" + "="*80)
print("🚀 TRAINING MODEL ON VEGETABLES")
print("="*80)

steps_per_epoch = veg_train_gen.samples // PARAMS['batch_size']
validation_steps = veg_val_gen.samples // PARAMS['batch_size']

print(f"\n📊 Training Configuration:")
print(f"  Batch size: {PARAMS['batch_size']}")
print(f"  Steps per epoch: {steps_per_epoch}")
print(f"  Validation steps: {validation_steps}")
print(f"  Epochs: {PARAMS['epochs']}")

print(f"\n🚀 Starting training...")
print(f"{'='*80}")

history = model.fit(
    veg_train_gen,
    steps_per_epoch=steps_per_epoch,
    validation_data=veg_val_gen,
    validation_steps=validation_steps,
    epochs=PARAMS['epochs'],
    callbacks=callbacks,
    verbose=1
)

print(f"{'='*80}")
print(f"✅ Training complete!")



🚀 TRAINING MODEL ON VEGETABLES

📊 Training Configuration:
  Batch size: 32
  Steps per epoch: 468
  Validation steps: 93
  Epochs: 50

🚀 Starting training...
Epoch 1/50
468/468 [==============================] - ETA: 0s - loss: 3.1162 - accuracy: 0.0672
Epoch 1: val_accuracy improved from -inf to 0.06653, saving model to C:\Users\USER\fruit_veg_mushroom_identifier\models\vegetables_model_best.h5
468/468 [==============================] - 981s 2s/step - loss: 3.1162 - accuracy: 0.0672 - val_loss: 2.8545 - val_accuracy: 0.0665 - lr: 0.0010
Epoch 2/50
468/468 [==============================] - ETA: 0s - loss: 2.9014 - accuracy: 0.0712
Epoch 2: val_accuracy improved from 0.06653 to 0.08837, saving model to C:\Users\USER\fruit_veg_mushroom_identifier\models\vegetables_model_best.h5
468/468 [==============================] - 924s 2s/step - loss: 2.9014 - accuracy: 0.0712 - val_loss: 2.7211 - val_accuracy: 0.0884 - lr: 0.0010
Epoch 3/50
442/468 [===========================>..] - ETA: 43s - l

KeyboardInterrupt: 